## Knowledge Graph
#### Drug-Food or Drug-Supplements interaction prediction

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

### Split data to train and test datasets

In [2]:
data_dir = 'data/triplets/'

In [3]:
ddi_df = pd.read_csv(data_dir + 'ddi.tsv', sep='\t')

In [4]:
print('total interactions:', ddi_df.shape[0])
print('unique interactions:', len(set(ddi_df.interaction)))

ddi_df.groupby(by=['interaction']).size()

total interactions: 3123450
unique interactions: 205


interaction
decrease_absorption                             576
decrease_adverse_effects                        384
decrease_anticholinergic_activities              26
decrease_anticoagulant_activities              5174
decrease_antihypertensive_activities          63420
                                              ...  
increase_visual_accommodation_disturbances        2
increase_vomiting                                54
increase_water_intoxication                     256
increase_weakness                              1710
increase_weight_gain                             16
Length: 205, dtype: int64

In [5]:
drug_supplement_df = pd.read_csv(data_dir + 'ds_relations.tsv', sep='\t')
drug_supplement_df = drug_supplement_df[drug_supplement_df['REL'] != 'has_ingredient']
drug_supplement_df.head()

,Unnamed: 0,CUI1,REL,CUI2
689297,689297,DC0477348,has_adverse_effect_on,DC0478433
689298,689298,DC0477348,has_adverse_effect_on,DC0478434
689299,689299,DC0477348,has_adverse_effect_on,DC0478432
689300,689300,DC0477348,has_adverse_effect_on,DC0478435
689301,689301,DC0477348,interacts_with,DC0478436


In [18]:
print('total interactions:', drug_supplement_df.shape[0])
print('unique interactions:', len(set(drug_supplement_df.REL)))

drug_supplement_df.groupby(by=['REL']).size()

total interactions: 18867
unique interactions: 5


REL
has_adverse_effect_on    3120
has_adverse_reaction     2093
has_therapeutic_class    5443
interacts_with           3057
is_effective_for         5154
dtype: int64

In [ ]:
def split_data():
    